<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/HW2_%E7%A4%BE%E5%9C%98%E6%94%B6%E6%94%AF%E7%B4%80%E9%8C%84%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import gradio as gr

In [18]:
# Google Sheets 認證
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟試算表
gsheets = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1T_CYo5tydoyyBj8LfMpQCcAD4IJqnNkczZb_6WIslZk/edit?gid=0#gid=0'
)

# 取得或建立 worksheet
try:
    worksheet = gsheets.worksheet('收支紀錄')
except:
    worksheet.append_row(['日期', '項目名稱', '收入', '支出', '請款人', '支付方式', '是否已給錢', '結餘'])

In [19]:
def add_record(date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status):
    # 如果選 "其他"，就用 other_method 當支付方式
    if payment_method == "其他" and other_method.strip() != "":
        payment_method = other_method.strip()

    # 判斷收入/支出
    if in_or_out == "收入":
        income = float(amount)
        expense = 0
    else:
        expense = float(amount)
        income = 0

    # 新增資料
    new_row = [date_str, item_str, income, expense, payer, payment_method, paid_status, ""]
    worksheet.append_row(new_row)

    # 更新 DataFrame
    records = worksheet.get_all_records()
    df = pd.DataFrame(records)
    df.columns = df.columns.str.strip()
    df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
    df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)

    # 計算累積結餘
    df['結餘'] = (df['收入'] - df['支出']).cumsum()

    # 更新試算表結餘欄位
    cell_list = worksheet.range(f"H2:H{len(df)+1}")
    for i, cell in enumerate(cell_list):
        cell.value = int(df.iloc[i]['結餘'])
    worksheet.update_cells(cell_list)

    # 計算總結
    total_income = df['收入'].sum()
    total_expense = df['支出'].sum()
    balance = df['結餘'].iloc[-1] if not df.empty else 0

    summary = f"✅ 已新增一筆紀錄！\n\n"
    summary += f"📌 項目: {item_str}\n💰 收入: {income}\n💸 支出: {expense}\n👤 請款人: {payer}\n"
    summary += f"💳 支付方式: {payment_method}\n📅 日期: {date_str}\n\n"
    summary += "--- 收支總結 ---\n"
    summary += f"總收入: {total_income:,.0f}\n"
    summary += f"總支出: {total_expense:,.0f}\n"
    summary += f"目前結餘: {balance:,.0f}\n"

    # 未付款項目
    unpaid_msg = "\n--- 未付款項目 ---\n"
    if '是否已給錢' in df.columns:
        unpaid = df[df['是否已給錢'].str.contains("否", na=False)]
        if not unpaid.empty:
            unpaid_msg += unpaid[['日期', '項目名稱', '支出', '請款人', '支付方式']].to_string(index=False)
        else:
            unpaid_msg += "✅ 沒有未付款項目"
    else:
        unpaid_msg += "❌ 試算表沒有「是否已給錢」欄位"

    return summary + "\n" + unpaid_msg

In [20]:
# --- 新增的分析函式 ---
# 修復後的 generate_analysis_to_sheet 函式
def generate_analysis_to_sheet():
    ANALYSIS_WS_NAME = '分析總結'

    # 重新獲取 '收支紀錄' 工作表，確保變數有效
    try:
        # 假設 gsheets (試算表物件) 在全域範圍內是有效的
        source_worksheet = gsheets.worksheet('收支紀錄')
    except gspread.WorksheetNotFound:
        return "❌ 錯誤：找不到 '收支紀錄' 工作表。"
    except Exception as e:
        return f"❌ 連線到 '收支紀錄' 工作表時發生錯誤：{e}"

    # 1. 取得或建立「分析總結」工作表
    try:
        analysis_ws = gsheets.worksheet(ANALYSIS_WS_NAME)
    except gspread.WorksheetNotFound:
        analysis_ws = gsheets.add_worksheet(title=ANALYSIS_WS_NAME, rows="100", cols="20")

    # 2. 讀取並處理數據
    try:
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)
        df.columns = df.columns.str.strip()
        df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
        df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)
    except Exception as e:
        return f"❌ 讀取資料失敗：{e}"

    if df.empty:
        return f"⚠️ '收支紀錄' 工作表沒有任何資料。"

    # 3. 執行收支分析

    # a. 總結餘
    total_income = df['收入'].sum()
    total_expense = df['支出'].sum()
    current_balance = total_income - total_expense

    # b. 請款人支出排名
    payer_summary = df.groupby('請款人')['支出'].sum().sort_values(ascending=False).reset_index()
    payer_summary.columns = ['請款人', '總支出']

    # c. 支付方式支出分佈
    payment_summary = df.groupby('支付方式')['支出'].sum().sort_values(ascending=False).reset_index()
    payment_summary.columns = ['支付方式', '總支出']

    # 4. 寫入分析結果到「分析總結」工作表

    # 清空現有內容
    analysis_ws.clear()

    # 寫入總結 (從 A1 開始)
    summary_data = [
        [f'收支分析總覽 ({datetime.now().strftime("%Y-%m-%d %H:%M:%S")})', ''],
        ['總收入', total_income],
        ['總支出', total_expense],
        ['目前總結餘', current_balance]
    ]
    analysis_ws.update('A1', summary_data)

    # 寫入請款人支出排名 (從 A6 開始)
    payer_data = [['請款人支出排名', '']] + [payer_summary.columns.tolist()] + payer_summary.values.tolist()
    analysis_ws.update('A6', payer_data)

    # 寫入支付方式分佈 (從 E6 開始)
    payment_data = [['支付方式支出分佈', '']] + [payment_summary.columns.tolist()] + payment_summary.values.tolist()
    analysis_ws.update('E6', payment_data)

    # 寫入未付款項目列表 (從 A15 開始)
    unpaid = df[df['是否已給錢'].str.contains("否", na=False)]
    unpaid_data = [['未付款項目 (待結清)', '']]
    if not unpaid.empty:
        unpaid_df_data = unpaid[['日期', '項目名稱', '支出', '請款人']].sort_values(by='日期').values.tolist()
        unpaid_data += [['日期', '項目名稱', '支出', '請款人']] + unpaid_df_data

    analysis_ws.update('A15', unpaid_data)


    return f"✅ 分析結果已成功儲存到 **{ANALYSIS_WS_NAME}** 工作表！\n\n請打開 Google Sheet 查看分析報告。"

In [21]:
# Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("# 💵 收支管理系統（Google Sheets 版）")

    with gr.Row():
        date_str = gr.Textbox(label="日期 (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))
        item_str = gr.Textbox(label="項目名稱")

    with gr.Row():
        in_or_out = gr.Radio(["收入", "支出"], label="收支類型", value="支出")
        amount = gr.Number(label="金額", value=0)

    with gr.Row():
        payer = gr.Textbox(label="請款人")
        payment_method = gr.Dropdown(choices=["現金", "信用卡", "Line Pay", "其他"], label="支付方式", value="現金")
        other_method = gr.Textbox(label="若選擇其他，請輸入方式")
        paid_status = gr.Radio(["是", "否"], label="是否已給錢", value="是")

    submit_btn = gr.Button("新增紀錄")
    output = gr.Textbox(label="結果", lines=15)

    submit_btn.click(
        fn=add_record,
        inputs=[date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status],
        outputs=output
    )
    # --- 分析區塊 ---
    with gr.Tab("執行數據分析"):
        gr.Markdown("### 📊 收支數據分析與報告生成")
        gr.Markdown("點擊按鈕，系統將讀取所有收支紀錄，進行 **總結餘、請款人排名、支付方式分佈** 等分析，並將結果寫入名為 **`分析總結`** 的新工作表。")
        analysis_btn = gr.Button("執行分析並寫入 Google Sheet")
        analysis_output = gr.Textbox(label="分析結果狀態", lines=5)

        analysis_btn.click(
            fn=generate_analysis_to_sheet,
            inputs=[],
            outputs=analysis_output
        )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe38d2903ff604d9c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
